In [ ]:
import cobra
import pandas as pd
import os

In [180]:
def produce (table,t_id):
    direction = []
    for r in table.index:
        if t_id in [i.id for i in s12.reactions.get_by_id(r).products]:
            direction.append(1)
        else:
            direction.append(-1)
    table['direction'] = direction
    
    a = tmp[(tmp['direction']<0) & (tmp['fluxes']<0)]
    b = tmp[(tmp['direction']>0) & (tmp['fluxes']>0)]
    
    produce_table = pd.concat([a,b])[['fluxes']].abs()
    
    return produce_table

In [352]:
metabolites = ['glc__D','hpta','octanol','styrene']
simulation_result = {'growth':[],'total_atp':[],'reactions_atp':[],
                     'total_nadh':[],'reactions_nadh':[],'total_nadph':[],
                     'reactions_nadph':[],'byproducts':[]}
target = ['atp','nadh','nadph']

for m in metabolites:
    m_id = 'EX_'+m+'_e'
    #s12 = cobra.io.read_sbml_model(os.getcwd()+'\\input\\iSH1474.xml')
    s12 = cobra.io.read_sbml_model('iSH1474.xml')
    
    with s12:
        s12.reactions.get_by_id('EX_glc__D_e').bounds = (0,0)
        s12.reactions.get_by_id(m_id).bounds = (-10,0)
        
        simulation_result['growth'].append(s12.slim_optimize())
        
        for t in target:
            t_id = t+'_c'
            tmp = s12.optimize().to_frame().T[[i.id for i in s12.metabolites.get_by_id(t_id).reactions]].T
            
            produce_table = produce(tmp,t_id)
            
            # total produced flux
            total = produce_table.fluxes.sum()
            simulation_result['total_'+t].append(total)
            
            # producing reactions
            cumsum = produce_table.sort_values('fluxes',ascending=False).cumsum()
            simulation_result['reactions_'+t].append(cumsum[cumsum['fluxes']<total*0.9].index.to_list())
            
        # byproduct
        tmp = s12.summary(solution=s12.optimize()).secretion_flux
        byproduct = tmp[(tmp['flux']<0) & (tmp['reaction'].str.contains('EX_'))].metabolite.to_list()
        if len(byproduct) != 0:
            simulation_result['byproducts'].append(byproduct)
        else:
            simulation_result['byproducts'].append('None')

df=pd.DataFrame(simulation_result, index=metabolites)
display(df)

,growth,total_atp,reactions_atp,total_nadh,reactions_nadh,total_nadph,reactions_nadph,byproducts
glc__D,0.992312,72.115711,"[ATPS4rpp, PGK]",24.977916,"[PDH, MDH, AKGDH]",13.907251,"[GAPDi_nadp, ICDHyr]","[co2_e, h2o_e, h_e]"
hpta,0.667678,60.580270,"[ATPS4rpp, PPK]",20.062142,"[MDH, GAPD, AKGDH]",15.234400,"[3HPTNCOAR, RHACOAR70, G3PD2]","[ac_e, co2_e, h2o_e, h_e]"
octanol,0.702013,66.637941,"[ATPS4rpp, PPK]",34.531569,"[MDH, OCTANOLD, ALDD8, HACD1i, HACD3i]",17.836090,"[ATHRDHr, G3PD2]","[ac_e, co2_e, h2o_e, h_e]"
styrene,0.524327,48.617074,"[ATPS4rpp, SUCOAS, PPK]",21.768035,"[MDH, HADPCOADH3, ALDD19xr]",9.942697,"[OXCOAHDH, ICDHyr, ME2]","[co2_e, h2o_e, h_e]"
